In [1]:
%load_ext watermark
%watermark

Last updated: 2021-06-02T10:26:47.744827-05:00

Python implementation: CPython
Python version       : 3.7.6
IPython version      : 7.12.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
import pandas as pd

In [3]:
datos = pd.read_csv("data/titanic.csv")

In [4]:
datos.head()

,superviviente,clase_billete,genero,edad,n_hermanos_esposos,n_hijos_padres,precio_billete,puerto_salida
0,0,3,hombre,22.0,1,0,7.2500,S
1,1,1,mujer,38.0,1,0,71.2833,C
2,1,3,mujer,26.0,0,0,7.9250,S
3,1,1,mujer,35.0,1,0,53.1000,S
4,0,3,hombre,35.0,0,0,8.0500,S


In [5]:
from sklearn import tree
from sklearn.model_selection import cross_val_score

In [6]:
arbol = tree.DecisionTreeClassifier()

In [7]:
from sklearn import preprocessing

In [8]:
columnas_categoricas = ["genero", "puerto_salida"]

In [9]:
codificador_binario = preprocessing.LabelBinarizer()

In [10]:
datos_categoricos = pd.get_dummies(datos[columnas_categoricas])

In [11]:
datos_categoricos

,genero_hombre,genero_mujer,puerto_salida_C,puerto_salida_Q,puerto_salida_S
0,1,0,0,0,1
1,0,1,1,0,0
2,0,1,0,0,1
3,0,1,0,0,1
4,1,0,0,0,1
...,...,...,...,...,...
886,1,0,0,0,1
887,0,1,0,0,1
888,0,1,0,0,1
889,1,0,1,0,0


In [12]:
pasajeros = (
    pd.concat([
        datos.drop(columnas_categoricas, axis=1),
        datos_categoricos
    ],axis=1
    )
)
pasajeros.edad = pasajeros.edad.fillna(pasajeros.edad.mean())

In [13]:
pasajeros.head()

,superviviente,clase_billete,edad,n_hermanos_esposos,n_hijos_padres,precio_billete,genero_hombre,genero_mujer,puerto_salida_C,puerto_salida_Q,puerto_salida_S
0,0,3,22.0,1,0,7.2500,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,0,1,1,0,0
2,1,3,26.0,0,0,7.9250,0,1,0,0,1
3,1,1,35.0,1,0,53.1000,0,1,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,0,0,1


In [14]:
arbol.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [15]:
%matplotlib inline

In [16]:
import graphviz

In [17]:
def dibujar_arbol(arbol):
    dot_data = tree.export_graphviz(arbol, out_file=None, 
                         feature_names=pasajeros.drop("superviviente", axis=1).columns,  
                         filled=True, 
                         impurity=False,
                         rounded=True,  
                         special_characters=True)  
    
    graph = graphviz.Source(dot_data)
    graph.format = 'png'
    graph.render('arbol',view=True)

In [18]:
dibujar_arbol(arbol)

Una funcionalidad interesante que tienen los arboles de decisión en sklearn es que nos dan una indicación de la importancia de cada variable en el modelo, almacenada en el atributo feature_importances_. Calcula la importancia en función de la ganancia de información de cada variable, es decir, que variables separan mejor las distintas clases. 

In [19]:
arbol.feature_importances_

array([0.10653511, 0.25342673, 0.05297195, 0.01736452, 0.24091175,
       0.30933519, 0.        , 0.00337536, 0.00331479, 0.01276461])

In [20]:
dict(zip(
    pasajeros.drop("superviviente", axis=1),
    arbol.feature_importances_
))

{'clase_billete': 0.10653510821979739,
 'edad': 0.2534267276251307,
 'n_hermanos_esposos': 0.05297195081482074,
 'n_hijos_padres': 0.017364522327815764,
 'precio_billete': 0.24091175107401014,
 'genero_hombre': 0.30933518862833875,
 'genero_mujer': 0.0,
 'puerto_salida_C': 0.0033753580690454106,
 'puerto_salida_Q': 0.0033147880984040054,
 'puerto_salida_S': 0.012764605142637166}

In [21]:
tree.DecisionTreeClassifier?

In [22]:
arbol_simple =tree.DecisionTreeClassifier(max_depth=3)

In [23]:
arbol_simple.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [25]:
dibujar_arbol(arbol_simple)

In [26]:
cross_val_score(arbol_simple, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc", cv=10).mean()

0.8502134227428346

In [27]:
arbol_balanceado = tree.DecisionTreeClassifier(max_depth=3, class_weight="balanced")

In [28]:

arbol_balanceado.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [29]:
dibujar_arbol(arbol_balanceado)

In [30]:
cross_val_score(arbol_balanceado, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc", cv=10).mean()

0.8615064652123475

In [31]:
cross_val_score(arbol_balanceado, pasajeros.drop("superviviente", axis=1), pasajeros.superviviente,
                scoring="precision", cv=10).mean()

0.7411115910638909

In [32]:
arbol_aleatorio = tree.ExtraTreeClassifier(max_features=1)